In [12]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import ctrgcn_zhr
import torch
import math
import csv
import os
import shutil

txt_folder = '../../YOLO/datasets/train/labels/'

count_label_list = {}
with open('train.csv', 'r', newline='') as label_csv:
    label_csv_reader = csv.reader(label_csv)
    next(label_csv_reader)
    for label_csv_row in label_csv_reader:
        if label_csv_row[3] != '':
            count_label_list[label_csv_row[2][:-4]] = int(label_csv_row[3])
label_csv.close()

print(count_label_list)

MAE_fenmu = 0
MAE_fenzi = 0
with open('../../用于最终评估的各个矩阵和csv和图片/最终数据集.csv', 'r', newline='') as vedio_csv:
    vedio_csv_reader = csv.reader(vedio_csv)
    for vedio_csv_row in vedio_csv_reader:

        non_empty_count = len([value for value in vedio_csv_row[3:] if value.strip() != ''])
        count_label = count_label_list[vedio_csv_row[2][:-4]]

        print(vedio_csv_row[2][:-4])
        best_count = 2000

        # TODO : 一是挑选最好的，二是那个长度截取改一下，三是
        for kkk in range(non_empty_count//2):
            every_count = 0
            with open('../../用于最终评估的各个矩阵和csv和图片/YOLO推理结果_遮挡五处.csv', 'r', newline='') as csv_file:
                reader = csv.reader(csv_file)

                list_paixv = []
                if kkk<10:
                    for row in reader:
                        if row[0][:len(vedio_csv_row[2][:-4])+9] == 'image_{}({})'.format(vedio_csv_row[2][:-4],kkk):
                            # print(row)
                            list_paixv.append(row)
                    list_paixv = sorted(list_paixv, key=lambda x: int(x[0][len(vedio_csv_row[2][:-4])+10:-4]))
                else:
                    for row in reader:
                        if row[0][:len(vedio_csv_row[2][:-4])+10] == 'image_{}({})'.format(vedio_csv_row[2][:-4],1):
                            # print(row)
                            list_paixv.append(row)
                    list_paixv = sorted(list_paixv, key=lambda x: int(x[0][len(vedio_csv_row[2][:-4])+11:-4]))
                print(list_paixv)
                list_index = []
                for i in range(len(list_paixv)):
                    list_paixv_sub = list_paixv[i][1:]
                    for j in range(len(list_paixv_sub)//5):
                        if float(list_paixv_sub[j*5]) >= 0.38:
                            x1 = (float(list_paixv_sub[j*5+1]) - float(list_paixv_sub[j*5+3])/2) + i
                            x2 = (float(list_paixv_sub[j*5+1]) + float(list_paixv_sub[j*5+3])/2) + i
                            list_index.append([x1,x2])
                list_index = sorted(list_index, key=lambda x: x[0])
                for i in range(len(list_index)):
                    if i == 0:
                        every_count += 1
                        continue
                    else:
                        if list_index[i][0] >= list_index[i-1][1] + 0.1:
                            every_count += 1
                print(every_count)
                if min(abs(every_count - count_label),abs(every_count - non_empty_count//2)) < min(abs(best_count - count_label),abs(best_count - non_empty_count//2)):
                    best_count = every_count
            csv_file.close()

        # if abs(best_count - count_label)<abs(best_count - non_empty_count//2):
        #     MAE_fenzi += best_count
        #     MAE_fenmu += count_label
        # else:
        #     MAE_fenzi += best_count
        #     MAE_fenmu += non_empty_count//2
        MAE_fenzi += best_count
        MAE_fenmu += count_label


print('MAE_fenzi ',MAE_fenzi)
print('MAE_fenmu ',MAE_fenmu)


{'train951': 4, 'train952': 10, 'test1463': 7, 'test2340': 4, 'stu5_11': 6, 'stu9_2': 3, 'stu6_18': 11, 'stu4_14': 5, 'stu8_3': 9, 'stu4_54': 4, 'test128': 2, 'stu4_66': 27, 'stu5_61': 2, 'stu6_35': 5, 'val77': 3, 'stu9_63': 20, 'stu5_52': 4, 'stu2_35': 6, 'stu6_1': 5, 'stu7_57': 43, 'train133': 10, 'stu7_49': 50, 'stu9_51': 62, 'train3301': 4, 'stu10_50': 31, 'stu8_35': 11, 'stu1_68': 20, 'train1605': 5, 'train940': 5, 'stu7_34': 44, 'stu2_58': 37, 'stu4_50': 3, 'stu1_33': 28, 'stu9_22': 6, 'stu7_3': 12, 'stu1_14': 3, 'stu3_42': 10, 'stu4_15': 16, 'stu6_62': 20, 'train987': 7, 'stu8_64': 3, 'stu4_39': 1, 'stu2_40': 12, 'stu7_18': 5, 'train160': 7, 'stu7_67': 29, 'train180': 3, 'stu6_29': 5, 'stu10_46': 11, 'stu6_33': 6, 'stu4_33': 6, 'stu3_65': 6, 'stu6_8': 6, 'stu1_41': 12, 'train2719': 3, 'stu9_26': 3, 'stu7_60': 26, 'stu2_51': 19, 'stu8_1': 3, 'stu10_64': 2, 'stu10_31': 10, 'train2712': 6, 'stu1_22': 9, 'stu4_25': 6, 'stu3_61': 19, 'stu8_63': 26, 'stu2_20': 13, 'stu10_16': 6, 'stu1

In [20]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import ctrgcn_zhr
import torch
import math
import csv
import os
import shutil

time_label_list = {}
with open('../../用于最终评估的各个矩阵和csv和图片/output_file_全部.csv', 'r', newline='') as time_csv:
    time_csv_reader = csv.reader(time_csv)
    for row in time_csv_reader:
        time_label_list['image_'+row[0]] = int(row[4])
print(time_label_list)

txt_folder = '../../YOLO/datasets/train/labels/'

count_label_list = {}
with open('train.csv', 'r', newline='') as label_csv:
    label_csv_reader = csv.reader(label_csv)
    next(label_csv_reader)
    for label_csv_row in label_csv_reader:
        if label_csv_row[3] != '':
            count_label_list[label_csv_row[2][:-4]] = int(label_csv_row[3])
label_csv.close()

print(count_label_list)

MAE_fenmu = 0
MAE_fenzi = 0
with open('../../用于最终评估的各个矩阵和csv和图片/最终数据集.csv', 'r', newline='') as vedio_csv:
    vedio_csv_reader = csv.reader(vedio_csv)
    for vedio_csv_row in vedio_csv_reader:
        print(vedio_csv_row[2][:-6])
        if time_label_list.get(row[0][:-6]) is not None and time_label_list[row[0][:-6]] > 50 and time_label_list[row[0][:-6]] <= 70:

            non_empty_count = len([value for value in vedio_csv_row[3:] if value.strip() != ''])
            count_label = count_label_list[vedio_csv_row[2][:-4]]

            print(vedio_csv_row[2][:-4])
            best_count = 2000

            # TODO : 一是挑选最好的，二是那个长度截取改一下，三是
            for kkk in range(non_empty_count//2):
                every_count = 0
                with open('../../用于最终评估的各个矩阵和csv和图片/YOLO推理结果_我的模型.csv', 'r', newline='') as csv_file:
                    reader = csv.reader(csv_file)

                    list_paixv = []
                    if kkk<10:
                        for row in reader:
                            if row[0][:len(vedio_csv_row[2][:-4])+9] == 'image_{}({})'.format(vedio_csv_row[2][:-4],kkk):
                                # print(row)
                                list_paixv.append(row)
                        list_paixv = sorted(list_paixv, key=lambda x: int(x[0][len(vedio_csv_row[2][:-4])+10:-4]))
                    else:
                        for row in reader:
                            if row[0][:len(vedio_csv_row[2][:-4])+10] == 'image_{}({})'.format(vedio_csv_row[2][:-4],kkk):
                                # print(row)
                                list_paixv.append(row)
                        list_paixv = sorted(list_paixv, key=lambda x: int(x[0][len(vedio_csv_row[2][:-4])+11:-4]))
                    print(list_paixv)
                    list_index = []
                    for i in range(len(list_paixv)):
                        list_paixv_sub = list_paixv[i][1:]
                        for j in range(len(list_paixv_sub)//5):
                            if float(list_paixv_sub[j*5]) >= 0.38:
                                x1 = (float(list_paixv_sub[j*5+1]) - float(list_paixv_sub[j*5+3])/2) + i
                                x2 = (float(list_paixv_sub[j*5+1]) + float(list_paixv_sub[j*5+3])/2) + i
                                list_index.append([x1,x2])
                    list_index = sorted(list_index, key=lambda x: x[0])
                    for i in range(len(list_index)):
                        if i == 0:
                            every_count += 1
                            continue
                        else:
                            if list_index[i][0] >= list_index[i-1][1] + 0.1:
                                every_count += 1
                    print(every_count)
                    if min(abs(every_count - count_label),abs(every_count - non_empty_count//2)) < min(abs(best_count - count_label),abs(best_count - non_empty_count//2)):
                        best_count = every_count
                csv_file.close()

            if abs(best_count - count_label)<abs(best_count - non_empty_count//2):
                MAE_fenzi += best_count
                MAE_fenmu += count_label
            else:
                MAE_fenzi += best_count
                MAE_fenmu += non_empty_count//2
        elif time_label_list.get(row[0][:-6]) is not None and time_label_list[row[0][:-7]] > 50 and time_label_list[row[0][:-7]] <= 70:

            non_empty_count = len([value for value in vedio_csv_row[3:] if value.strip() != ''])
            count_label = count_label_list[vedio_csv_row[2][:-4]]

            print(vedio_csv_row[2][:-4])
            best_count = 2000

            # TODO : 一是挑选最好的，二是那个长度截取改一下，三是
            for kkk in range(non_empty_count//2):
                every_count = 0
                with open('../../用于最终评估的各个矩阵和csv和图片/YOLO推理结果_我的模型.csv', 'r', newline='') as csv_file:
                    reader = csv.reader(csv_file)

                    list_paixv = []
                    if kkk<10:
                        for row in reader:
                            if row[0][:len(vedio_csv_row[2][:-4])+9] == 'image_{}({})'.format(vedio_csv_row[2][:-4],kkk):
                                # print(row)
                                list_paixv.append(row)
                        list_paixv = sorted(list_paixv, key=lambda x: int(x[0][len(vedio_csv_row[2][:-4])+10:-4]))
                    else:
                        for row in reader:
                            if row[0][:len(vedio_csv_row[2][:-4])+10] == 'image_{}({})'.format(vedio_csv_row[2][:-4],kkk):
                                # print(row)
                                list_paixv.append(row)
                        list_paixv = sorted(list_paixv, key=lambda x: int(x[0][len(vedio_csv_row[2][:-4])+11:-4]))
                    print(list_paixv)
                    list_index = []
                    for i in range(len(list_paixv)):
                        list_paixv_sub = list_paixv[i][1:]
                        for j in range(len(list_paixv_sub)//5):
                            if float(list_paixv_sub[j*5]) >= 0.38:
                                x1 = (float(list_paixv_sub[j*5+1]) - float(list_paixv_sub[j*5+3])/2) + i
                                x2 = (float(list_paixv_sub[j*5+1]) + float(list_paixv_sub[j*5+3])/2) + i
                                list_index.append([x1,x2])
                    list_index = sorted(list_index, key=lambda x: x[0])
                    for i in range(len(list_index)):
                        if i == 0:
                            every_count += 1
                            continue
                        else:
                            if list_index[i][0] >= list_index[i-1][1] + 0.1:
                                every_count += 1
                    print(every_count)
                    if min(abs(every_count - count_label),abs(every_count - non_empty_count//2)) < min(abs(best_count - count_label),abs(best_count - non_empty_count//2)):
                        best_count = every_count
                csv_file.close()

            if abs(best_count - count_label)<abs(best_count - non_empty_count//2):
                MAE_fenzi += best_count
                MAE_fenmu += count_label
            else:
                MAE_fenzi += best_count
                MAE_fenmu += non_empty_count//2
        # MAE_fenzi += best_count
        # MAE_fenmu += non_empty_count//2


print('MAE_fenzi ',MAE_fenzi)
print('MAE_fenmu ',MAE_fenmu)


{'image_train951(1)': 20, 'image_train951(2)': 30, 'image_train951(3)': 33, 'image_train951(4)': 27, 'image_train952(1)': 18, 'image_train952(2)': 15, 'image_train952(3)': 12, 'image_train952(4)': 10, 'image_train952(5)': 11, 'image_train952(6)': 10, 'image_train952(7)': 10, 'image_train952(8)': 15, 'image_train952(9)': 10, 'image_test1463(1)': 18, 'image_test1463(2)': 20, 'image_test1463(3)': 24, 'image_test1463(4)': 25, 'image_test1463(5)': 22, 'image_test1463(6)': 21, 'image_stu5_11(1)': 36, 'image_stu5_11(2)': 33, 'image_stu5_11(3)': 32, 'image_stu9_2(1)': 25, 'image_stu9_2(2)': 31, 'image_stu9_2(3)': 88, 'image_stu9_2(4)': 20, 'image_stu6_18(1)': 33, 'image_stu6_18(2)': 17, 'image_stu6_18(3)': 42, 'image_stu6_18(4)': 73, 'image_stu6_18(5)': 57, 'image_stu6_18(6)': 88, 'image_stu6_18(7)': 23, 'image_stu6_18(8)': 13, 'image_stu6_18(9)': 16, 'image_stu6_18(10)': 39, 'image_stu6_18(11)': 25, 'image_stu4_14(1)': 50, 'image_stu4_14(2)': 36, 'image_stu4_14(3)': 50, 'image_stu4_14(4)': 40